In [2]:
from environments.pickplace_environment_residual import ResPickOrPlaceEnvWithoutLangReward
import numpy as np
import matplotlib.pyplot as plt
from tasks.task import PutBlockInBowl,PickBowl,PutBowlInBowl
from stable_baselines3 import SAC,PPO,TD3
from agents.LLMRL import LLMSAC
import torch as th
from gymnasium import spaces


pybullet build time: May 20 2022 19:44:17
2023-10-24 09:55:46.493733: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-24 09:55:46.523639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-24 09:55:47.390017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
model =LLMSAC.load("tmp/llmsac_imgdrop15.0fixedputblockbowl_model/rl_model_80000_steps.zip")

input_dim: 145 14
input_dim: 145 14
input_dim: 145 14


/media/marunyu/Data/study/DELFT/master_thesis/RA/venv/lib/python3.8/site-packages/stable_baselines3/common/buffers.py:590: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 8.04GB > 7.02GB
  warnings.warn(


In [4]:
Env_num = 10
Envs = []
seed = [i for i in range(Env_num)]
for i in range(Env_num):
    np.random.seed(seed[i])
    env = ResPickOrPlaceEnvWithoutLangReward(
    image_obs=True,
    residual=True,
    render=False,
    multi_discrete=False,
    scale_action=True,
    ee ="suction",
    scale_obs=True,
    one_hot_action = True)
    Envs.append(env)


In [5]:
Obs = []
goals = []
for i in range(10):
    print("reset",i,"th env")
    obs,_ = Envs[i].reset()
    Obs.append(obs)
    goals.append(Envs[i].lang_goal)
    Envs[i].close()

reset 0 th env
init scene
obj_ids [6, 7, 8]
lang goal put the green block in a green bowl
reset 1 th env
init scene
obj_ids [6, 7, 8]
lang goal put the blue block in a yellow bowl
reset 2 th env
init scene
obj_ids [6, 7, 8]
lang goal put the yellow block in a blue bowl
reset 3 th env
init scene
obj_ids [6, 7, 8]
lang goal put the yellow block in a green bowl
reset 4 th env
init scene
init scene
obj_ids [6, 7, 8]
lang goal put the yellow block in a yellow bowl
reset 5 th env
init scene
obj_ids [6, 7, 8]
lang goal put the yellow block in a yellow bowl
reset 6 th env
init scene
obj_ids [6, 7, 8]
lang goal put the blue block in a blue bowl
reset 7 th env
init scene
init scene
obj_ids [6, 7, 8]
lang goal put the blue block in a yellow bowl
reset 8 th env
init scene
obj_ids [6, 7, 8]
lang goal put the green block in a blue bowl
reset 9 th env
init scene
obj_ids [6, 7, 8]
lang goal put the yellow block in a blue bowl


In [6]:
import ipywidgets
def plot_observation(env_id):
    print("goal:",goals[env_id])
    plt.subplot(1,2,1)
    plt.imshow(Obs[env_id]['rgbd'][:,:,:3])
    plt.subplot(1,2,2)
    plt.imshow(Obs[env_id]['rgbd'][:,:,3])
ipywidgets.interact(lambda env_id: plot_observation(env_id), env_id=(0,Env_num-1))

interactive(children=(IntSlider(value=4, description='env_id', max=9), Output()), _dom_classes=('widget-intera…

<function __main__.<lambda>(env_id)>

In [11]:
import copy
def feature_extract(obs):
    model.actor.set_training_mode(False)

    observation, vectorized_env = model.actor.obs_to_tensor(obs)
    extract_feature = model.policy.actor.extract_features(observation,model.policy.actor.features_extractor)
    return extract_feature

def show_feature(env_id,noise,position_noise,mask_goal = False):
    
    obs = copy.deepcopy(Obs[env_id])
    obs['rgbd'][:,:,:3] += 0.5*noise * np.random.randn(56,56,3)
    obs['rgbd'][:,:,3] += 0.5*noise * np.random.randn(56,56)
    obs["image"] += 0.5*position_noise * np.random.rand(obs["image"].shape[0])
    print("goal:",goals[env_id])
    plt.subplot(1,2,1)
    plt.imshow(obs['rgbd'][:,:,:3])
    plt.subplot(1,2,2)
    plt.imshow(obs['rgbd'][:,:,3])
    if mask_goal:
        print("mask_goal:",obs['lang_goal'])
        idx = obs['lang_goal'][0]
        obs['image'][idx*2:idx*2+2] = 0
    print("action:",np.round(model.predict(obs,deterministic = True)[0],2))
    # print("action:",model.policy.predict(obs,deterministic = True))
    # print("action:",model.policy.predict(obs),True)
    extract_feature = feature_extract(obs)
    print("extract_feature:",extract_feature)
ipywidgets.interact(lambda env_id,noise,position_noise: show_feature(env_id,noise,position_noise,1), env_id=(0,Env_num-1),noise=(0,1,0.1),position_noise=(0,1,0.1))

interactive(children=(IntSlider(value=4, description='env_id', max=9), FloatSlider(value=0.0, description='noi…

<function __main__.<lambda>(env_id, noise, position_noise)>